In [1]:
import geopandas as gpd

In [2]:
gdf = gpd.read_file('GMTED2010_Spatial_Metadata/GMTED2010_Spatial_Metadata.shp')

In [4]:
print(gdf.columns)
print(gdf.head())

Index(['ID', 'SOURCE_ORG', 'SOURCE', 'EL_SURFACE', 'NORTH', 'SOUTH', 'WEST',
       'EAST', 'X_SRCE_RES', 'Y_SRCE_RES', 'HORZ_UNIT', 'COORD_SYS',
       'HORZ_DATUM', 'VERT_DATUM', 'VERT_UNIT', 'MIN_ELEV', 'MAX_ELEV',
       'MEAN_ELEV', 'SDEV_ELEV', 'PROD_DATE', 'geometry'],
      dtype='object')
    ID       SOURCE_ORG                       SOURCE  EL_SURFACE  NORTH  \
0  268  Univ of Bristol  Ant Radar and Laser Alt DEM  Reflective    -55   
1  269  Univ of Bristol  Ant Radar and Laser Alt DEM  Reflective    -55   
2  270  Univ of Bristol  Ant Radar and Laser Alt DEM  Reflective    -55   
3  271  Univ of Bristol  Ant Radar and Laser Alt DEM  Reflective    -55   
4  272  Univ of Bristol  Ant Radar and Laser Alt DEM  Reflective    -55   

   SOUTH  WEST  EAST  X_SRCE_RES  Y_SRCE_RES  ...           COORD_SYS  \
0    -90  -180   180      1000.0      1000.0  ...  Polar Sterographic   
1    -90  -180   180      1000.0      1000.0  ...  Polar Sterographic   
2    -90  -180   180      1000.

In [16]:
import pandas as pd
from shapely.geometry import Point
from scipy.spatial import cKDTree
import numpy as np

df = pd.read_csv("LocationsWFire.csv")

elevations = []

# Create a KDTree for efficient nearest neighbor search
gdf_points = gdf['geometry'].apply(lambda geom: (geom.centroid.x, geom.centroid.y))
tree = cKDTree(gdf_points.tolist())

for i in range(len(df)):
    lat = df['latitude'][i]
    lon = df['longitude'][i]
    point = Point(lat, lon)
    mask = gdf['geometry'].contains(point)

    elevation_series = gdf.loc[mask, 'MEAN_ELEV']

    if elevation_series.empty:
        # Find the nearest point in gdf and use its elevation
        dist, idx = tree.query([lat, lon])
        elevation = gdf.iloc[idx]['MEAN_ELEV']
    else:
        elevation = elevation_series.values[0]

    elevations.append(elevation)

    print(f"{i}: {elevation}")

df['mean_elev'] = elevations
print(df.head())
df.to_csv("LocationsWFire_mean_elev.csv", index=False)

0: 251.518
1: 403.596
2: 117.016
3: 251.518
4: 14.378
5: 336.807
6: 403.596
7: 251.518
8: 403.596
9: 251.518
10: 336.807
11: 29.874
12: 29.874
13: 29.874
14: 117.016
15: 403.596
16: 117.016
17: 29.874
18: 29.874
19: 117.016
20: 14.378
21: 117.016
22: 29.874
23: 403.596
24: 29.874
25: 29.874
26: 14.378
27: 117.016
28: 251.518
29: 336.807
30: 14.378
31: 29.874
32: 29.874
33: 336.807
34: 336.807
35: 117.016
36: 403.596
37: 29.874
38: 29.874
39: 325.649
40: 336.807
41: 251.518
42: 251.518
43: 29.874
44: 403.596
45: 29.874
46: 29.874
47: 336.807
48: 29.874
49: 336.807
50: 403.596
51: 117.016
52: 336.807
53: 14.378
54: 403.596
55: 121.249
56: 14.378
57: 29.874
58: 336.807
59: 117.016
60: 25.022
61: 403.596
62: 25.022
63: 117.016
64: 25.022
65: 403.596
66: 14.378
67: 403.596
68: 251.518
69: 25.022
70: 29.874
71: 29.874
72: 1053.642
73: 117.016
74: 117.016
75: 336.807
76: 403.596
77: 25.022
78: 325.649
79: 403.596
80: 336.807
81: 336.807
82: 29.874
83: 336.807
84: 403.596
85: 29.874
86: 336.80

In [14]:
df['mean_elev'] = elevations
print(df.head())
df.to_csv("NaturalFiresTOPOGRAPHY_mean_elev.csv", index=False)

   year        date district municipality              parish  \
0  2001  2001-07-20     Faro    Monchique           Monchique   
1  2001  2001-07-15     Faro     Portimão  Mexilhoeira Grande   
2  2001  2001-07-02   Guarda      Gouveia          Folgosinho   
3  2001  2001-07-29   Guarda      Gouveia             Aldeias   
4  2001  2001-07-30   Guarda         Seia     Alvoco da Serra   

                  local   latitude  longitude    cause  elevation  ...  \
0       Castelo da Nave  37.283231  -8.577229  Natural    336.807  ...   
1        Pereira \ POVI  37.230364  -8.627352  Natural    336.807  ...   
2            FOLGOSINHO  40.511049  -7.512100  Natural     25.022  ...   
3            Taberneira  40.468718  -7.598221  Natural     25.022  ...   
4  Cerro dos Pardieiros  40.295477  -7.669545  Natural     25.022  ...   

  hourly.global_tilted_irradiance  hourly.terrestrial_radiation  \
0                           884.0                        1155.9   
1                           18

In [9]:
df.to_csv("NaturalFiresTOPOGRAPHY.csv", index=False)

In [7]:
from shapely.geometry import Point

point = Point(37.283231, -8.5772292)

mask = gdf['geometry'].contains(point)

elevation_series = gdf.loc[mask, 'MEAN_ELEV']

elevation = elevation_series.values[0]

print(elevation)

336.807


In [9]:
import rasterio

In [7]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from scipy.ndimage import generic_filter

In [12]:
with rasterio.open('md30_grd/md30_grd/w001001.adf') as src:
    raster_data = src.read(1)
    transform = src.transform

In [14]:
terrain = rd.rdarray(raster_data, no_data=src.nodata)

import numpy as np
from scipy.ndimage import generic_filter

def calculate_tri(dem):
    def tri_filter(x):
        central_value = x[len(x) // 2]
        return np.mean(np.abs(x - central_value))

    return generic_filter(dem, tri_filter, size=3)

# Calculate TRI
tri = calculate_tri(terrain)

# Define your coordinates
longitude = 37.283231  # replace with your longitude
latitude = -8.5772292  # replace with your latitude

# Convert the coordinate to raster indices
row, col = src.index(longitude, latitude)

# Get the TRI value at the coordinate
tri_value = tri[row, col]

print(tri_value)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7ff1c2d50af0>>
Traceback (most recent call last):
  File "/home/ori/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


: 

POLYGON ((-9.034817674180246 41.880570583659676, -8.67194576662672 42.13468943945496, -8.263856980817792 42.28046865495034, -8.013174607769912 41.790886135417125, -7.422512986673795 41.79207469335984, -7.251308966490824 41.91834605566505, -6.6686055159676565 41.883386949219584, -6.389087693700915 41.381815497394655, -6.851126674822552 41.11108266861753, -6.864019944679385 40.33087189387483, -7.026413133156595 40.184524237624245, -7.066591559263529 39.711891587882775, -7.498632371439726 39.62957103124181, -7.098036668313128 39.03007274022379, -7.374092169616318 38.37305858006492, -7.029281175148796 38.07576406508977, -7.166507941099865 37.803894354802225, -7.537105475281024 37.42890432387624, -7.453725551778092 37.09778758396607, -7.855613165711986 36.83826854099627, -8.382816127953689 36.97888011326246, -8.898856980820327 36.86880931248078, -8.746101446965554 37.65134552667661, -8.83999752443988 38.266243394517616, -9.287463751655224 38.3584858261586, -9.526570603869715 38.737429104154

/tmp/ipykernel_15252/4293272401.py:1: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [9]:
import rasterio
from rasterio.mask import mask
from scipy.ndimage import generic_filter

# Download the world countries shapefile
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Get the geometry of Portugal
portugal = world[world['name'] == 'Portugal']['geometry'].values[0]

with rasterio.open('md30_grd/md30_grd/w001001.adf') as src:
    # Clip the raster to the extent of Portugal
    out_image, out_transform = mask(src, [portugal.__geo_interface__], crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

# Save the clipped raster
with rasterio.open('clipped.tif', 'w', **out_meta) as dest:
    dest.write(out_image)

# Now you can calculate the TRI on the clipped raster
with rasterio.open('clipped.tif') as src:
    # Read the raster data
    raster_data = src.read(1)

    def calculate_tri(dem):
        def tri_filter(x):
            central_value = x[len(x) // 2]
            return np.mean(np.abs(x - central_value))

        return generic_filter(dem, tri_filter, size=3)

    # Calculate TRI
    tri = calculate_tri(raster_data)

/tmp/ipykernel_15252/815532472.py:6: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


ERROR! Session/line number was not unique in database. History logging moved to new session 553
